In [ ]:
# Importing needed packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load dataset
df = pd.read_csv("../input/household-power-consumption/household_power_consumption.txt", sep = ';', low_memory = False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'],na_values = '?', dtype = float)

In [ ]:
# View and confirm data rows and columns
df.shape

In [ ]:
df.dtypes

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

In [ ]:
# Finding null values
df.isnull().values.any()

In [ ]:
df.isnull().sum()

In [ ]:
df.replace('?', np.nan, inplace = True)

In [ ]:
#df.fillna(method= 'ffill', inplace= True)
data_df = df.apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
data_df.isnull().sum()

In [ ]:
# Downsample to daily data points
df_daily = data_df.resample('D').sum()
print(df_daily)

In [ ]:
import statsmodels.api as sm 
from pylab import rcParams 

In [ ]:
rcParams['figure.figsize'] = (15, 8) 
decompose_series = sm.tsa.seasonal_decompose(df_daily['Global_active_power'], model = 'additive') 
decompose_series.plot() 
plt.show()

In [ ]:
data1 = df['Global_active_power']
data2 = df['Global_reactive_power']

corr1 = data1.corr(data2)*100
round(corr1)

In [ ]:
data3 = df['Voltage']
data4 = df['Global_intensity']

corr2 = data3.corr(data4) * 100
round(corr2)

In [ ]:
from fbprophet import Prophet

In [ ]:
df_daily1 = df_daily.reset_index()
df_daily1.tail(365)

In [ ]:
train_dataset = df_daily1.rename(columns={"datetime": "ds", "Global_active_power": "y"})

In [ ]:
X = train_dataset[['ds','Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']]
y = train_dataset[['y']]

In [ ]:
train_dataset= pd.DataFrame()
train_dataset['ds'] = pd.to_datetime(X["ds"])
train_dataset['y'] = y

In [ ]:
train_dataset.head(12)

In [ ]:
train_dataset.tail(12)

In [ ]:
train_dataset.shape

In [ ]:
train_df = train_dataset[0:1077]
test_df = train_dataset[1077:]

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
model = Prophet() 
model.fit(train_df) 

In [ ]:
# Predict for the next year

future = model.make_future_dataframe(periods= 365)
forecast = model.predict(future)

In [ ]:
future.head()

In [ ]:
forecast.shape

In [ ]:
forecast.head() 

In [ ]:
forecast.tail() 

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']] 

In [ ]:
# yhat is the prediction while yhat_lower and yhat_upper are the upper and lower boundaries 
model.plot(forecast)
plt.show()

In [ ]:
import numpy as np

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
actual = test_df['y']
pred = forecast['yhat']
pred = pred[1077:]

In [ ]:
mape0 = mape(actual, pred)
round(mape0, 2)

In [ ]:
actual = test_df['y']
pred = forecast['yhat']
pred = pred[1077:]

In [ ]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(actual, pred)

rmse = np.sqrt(mse)

round(rmse, 2)

In [ ]:
fig1 = model.plot_components(forecast)

In [ ]:
train_dataset['Global_reactive_power'] = X['Global_reactive_power']
train_dataset['Voltage'] = X['Voltage']
train_dataset['Global_intensity'] = X['Global_intensity']
train_dataset['Sub_metering_1'] = X['Sub_metering_1']
train_dataset['Sub_metering_2'] = X['Sub_metering_2']
train_dataset['Sub_metering_3'] = X['Sub_metering_3']

train_df1 = train_dataset[0:1077]
test_df1 = train_dataset[1077:]

In [ ]:
train_dataset.head()

In [ ]:
pro_regressor = Prophet()
pro_regressor.add_regressor('Global_reactive_power')
pro_regressor.add_regressor('Voltage')
pro_regressor.add_regressor('Global_intensity')
pro_regressor.add_regressor('Sub_metering_1')
pro_regressor.add_regressor('Sub_metering_2')
pro_regressor.add_regressor('Sub_metering_3')

In [ ]:
pro_regressor.fit(train_df1)
future1 = pro_regressor.make_future_dataframe(periods= 365)

In [ ]:
future1.head()

In [ ]:
forecast1 = pro_regressor.predict(test_df1)

In [ ]:
forecast1.shape

In [ ]:
forecast1.head()

In [ ]:
forecast1.tail()

In [ ]:
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend', 'trend_lower', 'trend_upper']] 

In [ ]:
# yhat is the prediction while yhat_lower and yhat_upper are the upper and lower boundaries 
model.plot(forecast1)
plt.show()

In [ ]:
fig2 = pro_regressor.plot_components(forecast1)

In [ ]:
import numpy as np

def mape(actual1, pred1): 
    actual1, pred1 = np.array(actual1), np.array(pred1)
    return np.mean(np.abs((actual1 - pred1) / actual1)) * 100

In [ ]:
actual1 = test_df['y']
pred1 = forecast1['yhat']

In [ ]:
mape1 = mape(actual1, pred1)
round(mape1, 2)

In [ ]:
from sklearn.metrics import mean_squared_error
mse1 = mean_squared_error(actual1, pred1)

rmse1 = np.sqrt(mse1)

round(rmse1, 2)